importo il file json con i nomi, le coordinate e la pagina nel pdf delle mie tabelle 

In [1]:
import json

def coordinate():
    nomiT = []    # dichiaro l'array nomiT con i nomi delle Tabelle (tavole nel pdf) estrapolati da dati
    dati=[]       # dati conterrà i nomi + le coordinate della tabella 
    
    with open('coordinate.json') as json_file:
        datiCoordinate = json.load(json_file)

        dati = datiCoordinate
        print("***************************")

        for nome in datiCoordinate:
        
            nomiT.append(nome) 
    
    return(nomiT, dati)

acquisisco effettivamente i dati delle tabelle e li metto in df

Per via delle coordinate scelte nel pdf, non tutti i dati acquisiti sono davvero utili 

In [2]:
from tabula import read_pdf

var = coordinate()
df=[]

for i in range(0, len(var[0])):
    
    # [top,left,bottom,width]
    box = var[1][var[0][i]][0]      # contiene di volta in volta, le coordinate della tab
    pagina = var[1][var[0][i]][1]   # contiene la pagina della tab

    # fattore di conversione
    fc = 28.28
         
    for j in range(0, len(box)):
        box[j] /= 10
        box[j] *= fc
    
    df.append(read_pdf("dati/Utilizzo-e-qualità-della-risorsa-idrica-in-Italia.pdf", pages=pagina,area=[box],output_format="json", stream=True))

df    #contiene tutte le tabelle

# lo stampa in una maniera più facilmente leggibile ad un umano

***************************


[[{'bottom': 811.0704,
   'data': [[{'height': 6.830008506774902,
      'left': 42.52,
      'text': 'Tavola 1.1 – Prelievi di acqua per uso potabile per tipologia di fonte e regione. Anno 2015 (volumi in milioni di metri',
      'top': 420.81,
      'width': 422.8800048828125}],
    [{'height': 6.820000171661377,
      'left': 87.19,
      'text': 'cubi, pro capite in litri per abitante al giorno)',
      'top': 431.81,
      'width': 163.8199920654297}],
    [{'height': 13.059993743896484,
      'left': 43.52,
      'text': 'Prelevato pro REGIONI Sorgente Pozzo Corso d’acqua superfi ciale bacino artifi ciale Lago naturale o Acque marine o salmastre Totale capite',
      'top': 450.82,
      'width': 424.7099914550781}],
    [{'height': 4.659999847412109,
      'left': 43.51,
      'text': 'Piemonte  160,7  415,4  49,9  38,2 - 664,2 412',
      'top': 470.6,
      'width': 422.77001953125}],
    [{'height': 4.659999847412109,
      'left': 43.52,
      'text': 'Valle d’Aosta/Vallée d’

questa funzione, fa l'esportazione delle tabelle in un file json, con il rispettivo titolo 

In [3]:
def export(x,y):
    import json
    #import os
    with open("tabella1/"+x[0]+".json", "w") as outfile:
        json.dump(y, outfile, indent=3)
    
# il nome della tabella corrispode a quello sul pdf (Tavola *.*)
# alcune tabelle richiedono una revisione manuale del json (togliere 1 o 2 righe ad inizio tabella)
# esporto tutto su openrefine, divido i nomi dai valori e poi uso la RE di seguito --> (["   "]+|["  "]+|[" "]+)
# il risultato è la mia tabella corretta

Crea la tabella con i miei dati

******************************************************************************************************************

In [4]:
import re 

def tabella_fin():
    for j in range(0,len(df)):      # questo for mi permette di scorrere tutte le tabelle contenute in df     
        table = []
        for i in range(0,len(df[j][0]['data'])):
            if df[j][0]['data']:
                riga = df[j][0]['data'][i][0]['text']
                table.append(riga)  # aggiunge alla tabella le righe

        tab = []
        fin = []

        nome_tab = re.findall("[A-Za-z]+\s[0-9]+\.[0-9]+", table[0])  # contiene il nome tabella (es. Tavola 1.1)
        fonte_tab = table[len(df[j][0]['data'])-1]                    # contiene la fonte della tab (es istat)
        
#*****************************************************************************************************************                    
        if (nome_tab != [] or "") and (nome_tab != ['Tavola 3.2']):                # controlla che il nome sia presente 
            for i in range(1,len(table)):        #cicla partendo da uno, in modo da evitare il nome della tabella
                x = re.findall("([A-Za-z]+\s+)(-|([0-9]+|.,[0-9]+))", table[i])   # seleziona tutti i nomi (di regione) seguiti da numeri 
                if x:
                    tab.append(table[i])         # ora tab contien tutti i dati necessari estratti dal pdf, sono da sistemare

            for j in range(0,len(tab)):         # cicla dentro tab per poter sistemare le righe 
                y=re.search("([A-Za-z]\s+)(-|[0-9])", tab[j])   # trova tutti i nomi delle regioni + gli spazi + il primo numero
                if y:                          # toglie il priomo numero dal nome delle regioni ma lascia gli spazi
                    val = y.span()             # trova l'estensione della parola
                    nome = tab[j][0:val[1]-1]  # contiene il nome della regione (della riga)
                    nr = tab[j].split(nome)    # contiene il valore della riga (+ uno spazio bianco iniziale che sotto viene omesso)

                    riga_fin = str(nome)+":"+str(nr[1])      # è la riga che ottengo in definitiva 
                    fin.append(riga_fin)                    # aggiungo la riga a fin che restiuirò come risultato 
            
            print("*************************")
            print(nome_tab)           
#*****************************************************************************************************************                    
        else:                                  # mi gestisce la tavola 1.4 
            table = []
                
            for i in range(0,len(df[j][0]['data'])):
                if df[j][0]['data']:
                    riga = ""
                    for z in range(0, len(df[j][0]['data'][i])):      # serve a poter ciclare i valori singolarmente 
                        valore = df[j][0]['data'][i][z]['text']
                        if valore != " " or "":
                            riga += valore + "; "                    # diversamente d aprima, li separa con ;  
                table.append(riga)
    
            fonte_tab = table[len(df[j][0]['data'])-1]    # contiene la fonte della tab (es istat)
            #if nome_tab != ['Tavola 3.2']:              # volendo evita di aggiungere 'X9' a nome_tab
            nome_tab.append("X"+str(j))                 # diversamente da prima il titolo non è stato estratto dal pdf
            print ("tabella incriminata")              # la "j" che aggiungo ad X, si riferisce al primo ciclo for sopra 

            fin = table
            print("*************************")
            print(nome_tab)

#***************************************************************************************************************
        export(nome_tab,fin)      # fa l'esportazione in json della tabella, viene chiamato ad ogni ciclo del primo for iniziale 
    #return table

In [5]:
tabella_fin()

*************************
['Tavola 1.1']
*************************
['Tavola 1.2']
tabella incriminata
*************************
['X2']
*************************
['Tavola 1.5']
*************************
['Tavola 1.6']
*************************
['Tavola 1.8']
*************************
['Tavola 1.9']
*************************
['Tavola 1.10']
*************************
['Tavola 2.2']
tabella incriminata
*************************
['Tavola 3.2', 'X9']


Non ha molto senso dividere il numero dalla regione qui visto che posso usare open refine 
inoltre per poterla fare qui, dovrei fare una RE specifica che mi riconosca i numeri fatti in una certa maniera

a gruppi di tre, divisi da un punto, e l'ultimo gruppo diviso da una virgola, molto più facile far fare ad open refine 



*************************************************************************************